In [1]:
# IMPORT ALL YOUR LIBRARIES
!pip install transformers
!pip install datasets
!pip install accelerate >=0.21.0
!pip install transformers[torch]

import torch
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, Dataset
from datasets import Dataset
from transformers import AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import accuracy_score
from transformers import Trainer, TrainingArguments
import accelerate

import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from tqdm import tqdm
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
import matplotlib.pyplot as plt
import re

In [2]:
# !wget -P /kaggle/working -nc "https://raw.githubusercontent.com/MuhammadAreebKazmi2/Deep-Learning/refs/heads/main/Project/Dataset/TaskC/train/sun.csv"
!wget -P /kaggle/working -nc "https://raw.githubusercontent.com/MuhammadAreebKazmi2/Deep-Learning/refs/heads/main/Project/Dataset/track_a/train/deu.csv"
# !wget -P /kaggle/working -nc "https://raw.githubusercontent.com/HammadxSaj/SemEval_Task11_Dataset/refs/heads/main/deu_c.csv"
# !wget -P /kaggle/working -nc "https://raw.githubusercontent.com/MuhammadAreebKazmi2/Deep-Learning/refs/heads/main/Project/Dataset/TaskC/dev/sun_c.csv"
!wget -P /kaggle/working -nc "https://raw.githubusercontent.com/MuhammadAreebKazmi2/Deep-Learning/refs/heads/main/Project/Dataset/track_c/dev/rus.csv"

--2025-01-26 21:08:27--  https://raw.githubusercontent.com/MuhammadAreebKazmi2/Deep-Learning/refs/heads/main/Project/Dataset/track_a/train/deu.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 680115 (664K) [text/plain]
Saving to: ‘/kaggle/working/deu.csv’

deu.csv             100%[===================>] 664.17K  --.-KB/s    in 0.03s   

2025-01-26 21:08:27 (19.6 MB/s) - ‘/kaggle/working/deu.csv’ saved [680115/680115]

--2025-01-26 21:08:27--  https://raw.githubusercontent.com/MuhammadAreebKazmi2/Deep-Learning/refs/heads/main/Project/Dataset/track_c/dev/rus.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.1

In [3]:
df = pd.read_csv('/kaggle/working/deu.csv')
df.head()

,id,text,anger,disgust,fear,joy,sadness,surprise
0,deu_train_track_a_00001,"Hallo an alle Lesenden, da ich jetzt erstmal e...",0,0,1,0,0,0
1,deu_train_track_a_00002,Du warst bestimmt schon in verschiedenen Lände...,0,0,0,0,0,0
2,deu_train_track_a_00003,"wenn ihr um Geld spielt bei Bingo, fangen sie ...",0,0,0,0,0,0
3,deu_train_track_a_00004,Ich finde #Baerbock macht sich sehr gut als Au...,0,1,0,1,0,0
4,deu_train_track_a_00005,Ich war bei der zweiten Impfung bereits schwan...,0,0,0,0,0,0


In [5]:
!pip install accelerate

In [6]:
import pandas as pd
import torch
from transformers import SeamlessM4Tv2Model, AutoProcessor
from accelerate import infer_auto_device_map, dispatch_model

# Initialize Accelerate
from accelerate import Accelerator
accelerator = Accelerator()
device = accelerator.device

# Load the processor
processor = AutoProcessor.from_pretrained("facebook/seamless-m4t-v2-large")

# Load the model with an empty device map first
model = SeamlessM4Tv2Model.from_pretrained("facebook/seamless-m4t-v2-large", device_map="auto")

# Distribute the model across available GPUs
device_map = infer_auto_device_map(model)
model = dispatch_model(model, device_map=device_map)

# Define a function to translate text
def translate_text(text, src_lang, tgt_lang="eng"):
    try:
        # Process input and move tensors to device
        text_inputs = processor(text=text, src_lang=src_lang, return_tensors="pt").to(device)
        # Generate translation
        output_tokens = model.generate(**text_inputs, tgt_lang=tgt_lang, generate_speech=False)
        # Decode translation
        translated_text = processor.decode(output_tokens[0].tolist()[0], skip_special_tokens=True)
        return translated_text
    except Exception as e:
        print(f"Error translating text: {text}, Error: {e}")
        return text  # Return the original text if an error occurs


preprocessor_config.json:   0%|          | 0.00/1.78k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/19.7k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.17M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.07k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.34k [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/211k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.24G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/9.91M [00:00<?, ?B/s]

In [ ]:
# Load the CSV file
input_file = '/kaggle/working/deu.csv'
src_lang='deu'
data = pd.read_csv(input_file)

# Translate the 'text' column in small batches
batch_size = 8  # Adjust the batch size based on memory
translated_texts = []

for i in range(0, len(data), batch_size):
    batch = data['text'][i:i + batch_size]
    # Free GPU memory before processing each batch
    torch.cuda.empty_cache()
    translated_texts.extend([translate_text(text, src_lang) for text in batch])

# Replace the 'text' column with translated texts
data['text'] = translated_texts

# Save the translated file to a new CSV
output_file = '/kaggle/working/deu_translated.csv'
data.to_csv(output_file, index=False)

print(f"Translation completed. Translated file saved to {output_file}")


In [9]:
data

,id,text,anger,disgust,fear,joy,sadness,surprise
0,deu_train_track_a_00001,"Hello to all the readers, since I'm staying ho...",0,0,1,0,0,0
1,deu_train_track_a_00002,You've probably been to different countries be...,0,0,0,0,0,0
2,deu_train_track_a_00003,"When you play bingo for money, they start argu...",0,0,0,0,0,0
3,deu_train_track_a_00004,I think #Baerbock is doing very well as Foreig...,0,1,0,1,0,0
4,deu_train_track_a_00005,I was already pregnant with the second vaccina...,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
2598,deu_train_track_a_02599,My sister met her first boyfriend when she was...,0,0,0,0,1,0
2599,deu_train_track_a_02600,The grammar is questionable to the point of be...,0,0,0,0,0,0
2600,deu_train_track_a_02601,"Feelings are never ridiculous, so I have to di...",0,0,0,0,0,0
2601,deu_train_track_a_02602,When will the misery be over? Will they tear d...,0,0,0,0,1,0


In [11]:
!wget -P /kaggle/working -nc "https://raw.githubusercontent.com/MuhammadAreebKazmi2/Deep-Learning/refs/heads/main/Project/Dataset/track_c/dev/afr.csv"

--2025-01-26 22:54:35--  https://raw.githubusercontent.com/MuhammadAreebKazmi2/Deep-Learning/refs/heads/main/Project/Dataset/track_c/dev/afr.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12085 (12K) [text/plain]
Saving to: ‘/kaggle/working/afr.csv’

afr.csv             100%[===================>]  11.80K  --.-KB/s    in 0.001s  

2025-01-26 22:54:36 (17.9 MB/s) - ‘/kaggle/working/afr.csv’ saved [12085/12085]



In [12]:
# Load the CSV file
input_file = '/kaggle/working/afr.csv'
src_lang='afr'
data = pd.read_csv(input_file)

# Translate the 'text' column in small batches
batch_size = 8  # Adjust the batch size based on memory
translated_texts = []

for i in range(0, len(data), batch_size):
    batch = data['text'][i:i + batch_size]
    # Free GPU memory before processing each batch
    torch.cuda.empty_cache()
    translated_texts.extend([translate_text(text, src_lang) for text in batch])

# Replace the 'text' column with translated texts
data['text'] = translated_texts

# Save the translated file to a new CSV
output_file = '/kaggle/working/afr_c_translated.csv'
data.to_csv(output_file, index=False)

print(f"Translation completed. Translated file saved to {output_file}")


Translation completed. Translated file saved to /kaggle/working/afr_c_translated.csv


In [13]:
data

,id,text,anger,disgust,fear,joy,sadness
0,afr_dev_track_c_00001,Clear policy frameworks have already been laid...,0,0,0,1,0
1,afr_dev_track_c_00002,It's an industry that's 90% execution-oriented.,0,0,0,0,0
2,afr_dev_track_c_00003,In one of M.E.R.,0,0,0,0,0
3,afr_dev_track_c_00004,"So once you've reached the top, keep climbing.",0,0,0,1,0
4,afr_dev_track_c_00005,The cost is R1 135 000 and the capacity is inc...,0,0,0,1,0
...,...,...,...,...,...,...,...
93,afr_dev_track_c_00094,Because look how Keesje stares at him...,0,0,0,0,0
94,afr_dev_track_c_00095,And I am grateful that the same points were ma...,0,0,0,1,0
95,afr_dev_track_c_00096,This is well above the inflation target of bet...,0,0,1,0,1
96,afr_dev_track_c_00097,The African Patriot vigorously promoted this i...,0,0,0,0,0


In [14]:
!wget -P /kaggle/working -nc "https://raw.githubusercontent.com/MuhammadAreebKazmi2/Deep-Learning/refs/heads/main/Project/Dataset/track_c/dev/ary.csv"

--2025-01-26 23:02:33--  https://raw.githubusercontent.com/MuhammadAreebKazmi2/Deep-Learning/refs/heads/main/Project/Dataset/track_c/dev/ary.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 43246 (42K) [text/plain]
Saving to: ‘/kaggle/working/ary.csv’

ary.csv             100%[===================>]  42.23K  --.-KB/s    in 0.007s  

2025-01-26 23:02:34 (5.70 MB/s) - ‘/kaggle/working/ary.csv’ saved [43246/43246]



In [15]:
# Load the CSV file
input_file = '/kaggle/working/ary.csv'
src_lang='ary'
data = pd.read_csv(input_file)

# Translate the 'text' column in small batches
batch_size = 8  # Adjust the batch size based on memory
translated_texts = []

for i in range(0, len(data), batch_size):
    batch = data['text'][i:i + batch_size]
    # Free GPU memory before processing each batch
    torch.cuda.empty_cache()
    translated_texts.extend([translate_text(text, src_lang) for text in batch])

# Replace the 'text' column with translated texts
data['text'] = translated_texts

# Save the translated file to a new CSV
output_file = '/kaggle/working/ary_c_translated.csv'
data.to_csv(output_file, index=False)

print(f"Translation completed. Translated file saved to {output_file}")


Translation completed. Translated file saved to /kaggle/working/ary_c_translated.csv


In [16]:
data

,id,text,anger,disgust,fear,joy,sadness,surprise
0,ary_dev_track_c_00001,The Public Prosecutor's Office sent a group of...,0,0,0,0,0,1
1,ary_dev_track_c_00002,The corrupt state hopes that this demonstratio...,1,0,0,0,0,0
2,ary_dev_track_c_00003,@<username> @<username> hhhhhhhhhhh dahktinii ...,0,0,0,1,0,0
3,ary_dev_track_c_00004,Share this with your beloved father. May Allah...,0,0,0,0,1,0
4,ary_dev_track_c_00005,@<username> 60% kigolo howa ana l9ito msarab g...,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
262,ary_dev_track_c_00263,"Fearing for his life, he chose education to ru...",0,0,0,0,0,0
263,ary_dev_track_c_00264,A petition for three new players for the local...,0,0,0,0,0,0
264,ary_dev_track_c_00265,Hhhhhhhhhh zwina ahdi limchat lforossiya mchat...,0,0,0,1,0,0
265,ary_dev_track_c_00266,For the first time I will see a Moroccan YouTu...,0,0,0,1,0,0


In [17]:
!wget -P /kaggle/working -nc "https://raw.githubusercontent.com/MuhammadAreebKazmi2/Deep-Learning/refs/heads/main/Project/Dataset/track_a/train/ukr.csv"

--2025-01-26 23:09:34--  https://raw.githubusercontent.com/MuhammadAreebKazmi2/Deep-Learning/refs/heads/main/Project/Dataset/track_a/train/ukr.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 348190 (340K) [text/plain]
Saving to: ‘/kaggle/working/ukr.csv’

ukr.csv             100%[===================>] 340.03K  --.-KB/s    in 0.03s   

2025-01-26 23:09:34 (13.0 MB/s) - ‘/kaggle/working/ukr.csv’ saved [348190/348190]



In [ ]:
# Load the CSV file
input_file = '/kaggle/working/ukr.csv'
src_lang='ukr'
data = pd.read_csv(input_file)

# Translate the 'text' column in small batches
batch_size = 8  # Adjust the batch size based on memory
translated_texts = []

for i in range(0, len(data), batch_size):
    batch = data['text'][i:i + batch_size]
    # Free GPU memory before processing each batch
    torch.cuda.empty_cache()
    translated_texts.extend([translate_text(text, src_lang) for text in batch])

# Replace the 'text' column with translated texts
data['text'] = translated_texts

# Save the translated file to a new CSV
output_file = '/kaggle/working/ukr_c_translated.csv'
data.to_csv(output_file, index=False)

print(f"Translation completed. Translated file saved to {output_file}")
